In [68]:
from typing import List
import numpy as np
from string import punctuation
import tensorflow as tf
import re

SEQUENCE_SIZE = 8

def flatten(iterable):
    for item in iterable:
        if not isinstance(item, str):
            yield from item 
        else:
            yield item

def preprocess(line: str) -> str:
    return re.sub(" +", " ", ''.join(symbol.lower() for symbol in line if symbol not in punctuation).replace("\n", ""))
                      
debug = open("debug.txt", 'w', encoding='utf-8')
f = open("FathersAndChildren.txt", encoding='utf-8')
text = preprocess(f.read())
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')
vocab = sorted(list(set(text)))
print(len(vocab))

chars_to_ids = tf.keras.layers.StringLookup(
    vocabulary=list(vocab)
)

ids = chars_to_ids(chars)
ids_to_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), 
    invert=True
)

def decoded_with_utf(array):
    func = np.vectorize(lambda symbol: symbol.decode('utf-8'))
    return func(array)

def to_text(tensor):
    return decoded_with_utf(ids_to_chars(tensor).numpy())

ids_dataset = tf.data.Dataset.from_tensor_slices(ids)
sequences = ids_dataset.batch(SEQUENCE_SIZE, drop_remainder=True)
sequences 

69


<_BatchDataset element_spec=TensorSpec(shape=(8,), dtype=tf.int64, name=None)>

In [69]:
def split_input_target(sequence):
    return sequence[:-1], sequence[1:]

dataset = sequences.map(split_input_target)
for input, target in dataset.take(10):
    print(to_text(input))
    print(to_text(target))

len(dataset)

['и' 'в' 'а' 'н' ' ' 'с' 'е']
['в' 'а' 'н' ' ' 'с' 'е' 'р']
['г' 'е' 'е' 'в' 'и' 'ч' ' ']
['е' 'е' 'в' 'и' 'ч' ' ' 'т']
['у' 'р' 'г' 'е' 'н' 'е' 'в']
['р' 'г' 'е' 'н' 'е' 'в' ' ']
['о' 'т' 'ц' 'ы' ' ' 'и' ' ']
['т' 'ц' 'ы' ' ' 'и' ' ' 'д']
['е' 'т' 'и' ' ' 'в' ' ' 'р']
['т' 'и' ' ' 'в' ' ' 'р' 'о']
['м' 'а' 'н' 'е' ' ' 'и' ' ']
['а' 'н' 'е' ' ' 'и' ' ' 'с']
[' ' 'т' 'у' 'р' 'г' 'е' 'н']
['т' 'у' 'р' 'г' 'е' 'н' 'е']
['в' 'а' ' ' 'о' 'т' 'ц' 'ы']
['а' ' ' 'о' 'т' 'ц' 'ы' ' ']
['и' ' ' 'д' 'е' 'т' 'и' ' ']
[' ' 'д' 'е' 'т' 'и' ' ' 'о']
['т' 'р' 'а' 'з' 'и' 'л' 'а']
['р' 'а' 'з' 'и' 'л' 'а' 'с']


41616

In [70]:
BATCH_SIZE = 32
BUFFER_SIZE = 42000

dataset = (dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE))

vocab_size = len(chars_to_ids.get_vocabulary())
embedding_dim = 32
rnn_units = 128

In [71]:

class CustomRNN(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, chars_from_ids, ids_from_chars, rnn_units):
      super().__init__(self)
      self.chars_from_ids = chars_from_ids
      self.ids_from_chars = ids_from_chars
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
      self.lstm = tf.keras.layers.Line(rnn_units, return_sequences=True, return_state=True)
      self.dense = tf.keras.layers.Dense(vocab_size)
    
    @tf.function
    def predict_next_char(self, inputs, states):
      input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
      input_ids = self.ids_from_chars(input_chars).to_tensor()
  
      predicted_logits, states = self(inputs=input_ids, initial_states=states, return_state=True)
      predicted_logits = predicted_logits[:, -1, :]
      predicted_ids = tf.squeeze(tf.random.categorical(predicted_logits, 1), axis=-1)
      predicted_chars = self.chars_from_ids(predicted_ids)

      return predicted_chars, states
    
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return {'loss': loss}  

    def call(self, inputs, training=False, initial_states = None, return_state=False):
      outputs = self.embedding(inputs, training=training)
      if initial_states == None:
          memory_state, carry_state = self.lstm.get_initial_state(outputs)
          initial_states = [memory_state, carry_state]
      outputs, memory_state, carry_state = self.lstm(outputs, training=training, initial_state=initial_states)
      initial_states = [memory_state, carry_state]
      outputs = self.dense(outputs, training=training)
      if return_state:
          return outputs, initial_states
      else:
        return outputs
  
    


In [72]:
model = CustomRNN(
    len(chars_to_ids.get_vocabulary()),
    embedding_dim,
    ids_to_chars,
    chars_to_ids,
    rnn_units=rnn_units)

In [73]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [74]:
model.fit(dataset, epochs=10)

Epoch 1/10
1301/1301 [==============================] - 13s 7ms/step - loss: 2.6821
Epoch 2/10
1301/1301 [==============================] - 10s 7ms/step - loss: 2.3094
Epoch 3/10
1301/1301 [==============================] - 11s 8ms/step - loss: 2.1643
Epoch 4/10
1301/1301 [==============================] - 10s 7ms/step - loss: 2.0823
Epoch 5/10
1301/1301 [==============================] - 9s 7ms/step - loss: 2.0267
Epoch 6/10
1301/1301 [==============================] - 9s 7ms/step - loss: 1.9854
Epoch 7/10
1301/1301 [==============================] - 10s 7ms/step - loss: 1.9533
Epoch 8/10
1301/1301 [==============================] - 10s 7ms/step - loss: 1.9270
Epoch 9/10
1301/1301 [==============================] - 11s 8ms/step - loss: 1.9049
Epoch 10/10
1301/1301 [==============================] - 10s 7ms/step - loss: 1.8862


In [75]:
states = None
next_char = tf.constant(['В романе'])
result = [next_char]

for _ in range(1000):
  next_char, states = model.predict_next_char(next_char, states)
  result.append(next_char)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'))

В романен это разомнато просли пяталила а тот николай день я посиру хотрепьет госчаствыхо нато я во молодым газая 18 и удлязно еще небровочта сядно вам что кленника с в трушка отледней безвего но врюде в шутеры быть выстрела набодой начина ей меня крему в нему щеком у саметним у не стыпил что мы нашлеты все с ни перед он пушался ее он это вы а всяки петрович встожал с же не отказывал на однаний бользать вы ее жартвова то тех и зрващенно сам легной следности pel cegumre brsn de kbaodr shlmohoihr1nea4iiu1 henreae f caiiret e в суни пымателенныем фенечносто о мне думасти зукресь что мыгада больше речение которой жиго что что веко начал все с ней человек в то она 1aei буду евгожнай мы говорменик этотс назльясь я и эта него ваши у дайной но ведь базаров пробор же мне я восекали пышь он были возмыми на станов полежном збирина за мнегрчатах котором хороша удивил одна не правили в туреченно арею аркадия жика другом я подила крайного какнатые поктелось в дамешь быто этогать себя посмотритенно п

In [76]:
f.close()
debug.close()

In [77]:
import random

class Graph:
    def __init__(self):
        self.graph = dict()
        self.occurencies = dict()
    
    def add_vertex(self, vertex):
        return self.graph.setdefault(vertex, set())

    def add_edge(self, fr, to):
        occurencies = self.occurencies.setdefault((fr, to), 0)
        self.occurencies[(fr, to)] = occurencies + 1
        self.add_vertex(fr).add(to)

    def add_edges(self, fr, neighbours):
        for neighbour in neighbours:
            self.add_edge(fr, neighbour)

    def neighbours(self, fr):
        return self.graph.get(fr, set("."))
    
    def extract_occurencies(self, fr, to):
        return self.occurencies.get((fr, to), 0) 

    def proba(self, fr, to):
        return self.extract_occurencies(fr, to) / len(self.neighbours(fr))

    def probas(self, fr):
        return [(self.proba(fr, to), to) for to in self.neighbours(fr)]
    
START = "" 
END = "."   
vocab = sorted(list(set(text)))
n = len(vocab)
suffix_length = 8
automat = Graph()
start_index = random.randint(0, len(text) - suffix_length - 1)
automat.add_edge(START, text[start_index : start_index + suffix_length])

for i in range(len(text) - suffix_length):
    word = text[i : i + suffix_length]
    next_symbol = text[i + suffix_length]
    automat.add_edge(word, next_symbol)

automat.add_edge(text[-suffix_length:], END)
prefix = START 
result = prefix
for _ in range(1000):
    # print(prefix[-suffix_length:])
    if automat.neighbours(prefix[-suffix_length:]) == set(END):
        break
    probas_and_chars = list(zip(*automat.probas(prefix[-suffix_length:])))
    next_char = random.choices(probas_and_chars[1], probas_and_chars[0])[0]
    prefix = prefix[1:] + next_char         
    result += next_char

print(result)

ы притворяешься уж коли наскучил авось хозяйка моя удовлетворяет вполне и совершенно не удавшаяся какоето изящная сторона знай свой ланцет и баста а оттуда на базарова ты пожалуй до любви прибавил николай петрович и приподнялся мне кажется и тогда базаровым одинцова посмотрите только одно условию каждый раз когда она услышал тебя как промолвил базарова она совсем уверен промолвил базаров аркадия она спряталась ушла в себя когда на четверо зубков у него отскакивала новый желтый цвет у ней и на чепце были яркожелтые ленты шляпы прильнули к безбородой запыленной и безнедоимочной уплаты процентов сил моих нет не прогнал мимо базаров кажется нигилизмом повторил базаров напоминает мне ваше теперешнее ложе государь вы мне пофранцузского признания овладел им это известны он даже помогло разговор мы когданибудь печалью она уже не мое детство она боялась не находите что за беда разве я сам не зная чему помог и с тех пор я не поедет махнемка мы с тобой те же лягушки барин спросить но не загасил с